<a href="https://colab.research.google.com/github/shelly-hub/deep-learning-challenge/blob/main/Alphabet_soup_optimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Need to change value counts to list
app_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[app_counts < 500].index.tolist()
# print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_filterone = application_df['CLASSIFICATION'].value_counts()[class_counts > 1]
class_filterone

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[class_counts < 1000].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
# Make sure to change list dataframe to values
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns = 'IS_SUCCESSFUL').values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'tanh')
)
# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'tanh')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 300)

Epoch 1/300
804/804 [==============================] - 6s 5ms/step - loss: 0.5713 - accuracy: 0.7214
Epoch 2/300
804/804 [==============================] - 4s 5ms/step - loss: 0.5561 - accuracy: 0.7303
Epoch 3/300
804/804 [==============================] - 5s 6ms/step - loss: 0.5523 - accuracy: 0.7310
Epoch 4/300
804/804 [==============================] - 3s 4ms/step - loss: 0.5494 - accuracy: 0.7319
Epoch 5/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7319
Epoch 6/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7325
Epoch 7/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7341
Epoch 8/300
804/804 [==============================] - 3s 4ms/step - loss: 0.5461 - accuracy: 0.7354
Epoch 9/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7352
Epoch 10/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5694 - accuracy: 0.7250 - 586ms/epoch - 2ms/step
Loss: 0.5694196820259094, Accuracy: 0.7250145673751831


## **Model 2**

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =100
hidden_nodes_layer2 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu')
)
# Second hidden layer
nn2.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu')
)

# third hidden layer
nn2.add(
    tf.keras.layers.Dense(units = 10, activation = 'relu')
)


# Output layer
nn2.add(
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
)

# Check the structure of the model
nn2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,971
Trainable params: 9,971
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 8s 5ms/step - loss: 0.5671 - accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5533 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5515 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7326
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7352
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7354
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5453 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accura

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5670 - accuracy: 0.7263 - 741ms/epoch - 3ms/step
Loss: 0.5670210123062134, Accuracy: 0.7262973785400391


## **MODEL 3**

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =100
hidden_nodes_layer2 = 50

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu')
)
# Second hidden layer
nn3.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'tanh')
)

# third hidden layer
nn3.add(
    tf.keras.layers.Dense(units = 10, activation = 'tanh')
)

# Output layer
nn3.add(
    tf.keras.layers.Dense(units = 1, activation = 'relu')
)

# Check the structure of the model
nn3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,971
Trainable params: 9,971
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn3.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.7647 - accuracy: 0.6986
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5851 - accuracy: 0.7233
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5846 - accuracy: 0.7124
Epoch 4/100
804/804 [==============================] - 6s 8ms/step - loss: 0.5663 - accuracy: 0.7252
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5563 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5634 - accuracy: 0.7287
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5619 - accuracy: 0.7281
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5542 - accuracy: 0.7297
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5619 - accuracy: 0.7300
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6303 - accuracy: 0.7258 - 704ms/epoch - 3ms/step
Loss: 0.6303169131278992, Accuracy: 0.7258309125900269


In [17]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimisation.h5")